<a href="https://colab.research.google.com/github/nabadawy/DeepLearning/blob/master/Labs/Lab4/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment: Text Generation with RNNs

In this assignment, you will explore text generation using deep learning models, specifically Recurrent Neural Networks (RNNs). Text generation is a fascinating task where the goal is to train a model that can predict the next word in a sequence, ultimately generating coherent and contextually accurate sentences or paragraphs.

You will work with a corpus of text data provided in the tutorial (from TensorFlow), which contains a large collection of Shakespeare’s works. Your objective is to implement a text generation model using RNNs and to evaluate its performance based on various metrics.

<img src="https://drek4537l1klr.cloudfront.net/teofili2/Figures/03fig09_alt.jpg" alt="Drawing"/>

**The models include:**
- Deep RNN (LSTM)
- Deep RNN (GRU)
- Bidirectional RNN


**Evaluation:**

Evaluate the performance of the text generation model using several metrics:
- Perplexity: A measure of how well the probability distribution predicted by the model aligns with the actual distribution of words in the text.
- Generated Text Quality: Subjectively evaluate the quality of the generated text by considering grammar, coherence, and creativity. This can be done by visually inspecting the generated sequences.
- BLUE Score: Character-level BLEU: BLEU can be applied at the character level by treating each character as an n-gram. This is particularly useful when the task involves generating character sequences (like poetry, code, or fine-grained character-based generation).
For example, the sequence “hello” could be evaluated by comparing 1-grams (e.g., “h”, “e”, “l”, “l”, “o”) or higher-order n-grams (e.g., “he”, “el”, “ll”, “lo”).

**Goals:**

- Compare the performance of GRU and LSTM. Is there a significant difference in the results?
- Compare the performance of the unidirectional and bidirectional RNN models.
Which model produces better results?
- Discuss the impact of bidirectional RNNs on text generation tasks.

# Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf
import numpy as np
import nltk
from nltk.translate.bleu_score import sentence_bleu
import os
import time

# Dataset

The Shakespeare dataset used for text generation contains a collection of works by William Shakespeare, primarily in the form of plays and sonnets. The text is used to train language models, offering an ideal example for character-level modeling due to its rich, complex language. The dataset is often preprocessed by tokenizing it into individual characters, enabling models to learn the sequential relationships between characters for generating coherent and contextually appropriate text.

To access the dataset, you can use TensorFlow's get_file method as follow:

You can inspect the dataset on [Kaggle](https://www.kaggle.com/datasets/adarshpathak/shakespeare-text/data).

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



# Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation.

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>


Now create the [`tf.keras.layers.StringLookup`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup) layer:

Since the goal of this assignment is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use tf.keras.layers.StringLookup(..., invert=True).

Note: Here instead of passing the original vocabulary generated with sorted(set(text)) use the get_vocabulary() method of the tf.keras.layers.StringLookup layer so that the [UNK] tokens is set the same way.

In [ ]:
def get_string_lookup_layers(vocab):
    """
    Creates StringLookup layers for encoding characters to IDs and decoding IDs back to characters.

    Args:
        vocab (list): List of unique characters in the dataset.

    Returns:
        ids_from_chars (tf.keras.layers.StringLookup): Converts characters to IDs.
        chars_from_ids (tf.keras.layers.StringLookup): Converts IDs back to characters.
    """
    ids_from_chars = '''TO DO'''
    chars_from_ids = '''TO DO'''
    return ids_from_chars, chars_from_ids

In [ ]:
def text_from_ids(ids, chars_from_ids):
    """
    Converts a sequence of character IDs into a human-readable string.

    Args:
        ids (tf.Tensor): Tensor of character IDs.
        chars_from_ids (tf.keras.layers.StringLookup): StringLookup layer to decode IDs.

    Returns:
        tf.Tensor: Decoded string.
    """
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

# Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

For training you'll need a dataset of (input, label) pairs. Where input and label are sequences. At each time step the input is the current character and the label is the next character.

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

Convert Text to Numerical Sequences

In [ ]:
# Initialize mapping layers
vocab = sorted(set(text))
ids_from_chars, chars_from_ids = get_string_lookup_layers(vocab)



# Convert text to character IDs
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

# Create a dataset from the character IDs
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

Create Sequences for Training

In [ ]:
seq_length = 100  # Length of each training sequence

# Batch sequences (each sequence is seq_length + 1)
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

# Map dataset to input-target format
dataset = sequences.map(split_input_target)

# Print sample input-output pairs
for input_example, target_example in dataset.take(1):
    print("Input :", tf.strings.reduce_join(chars_from_ids(input_example)).numpy().decode('utf-8'))
    print("Target:", tf.strings.reduce_join(chars_from_ids(target_example)).numpy().decode('utf-8'))


Input : First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
Target: irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


# Create RNN Models

Before training our text generation models, we need to set up key parameters and prepare the dataset. These parameters control the size of batches, buffer size for shuffling, embedding dimensions, and the number of units in the recurrent layer.

Customizing Parameters for Optimization

- Increase RNN_UNITS if the model is underfitting (not capturing enough detail).
- Decrease BATCH_SIZE if memory usage is too high (e.g., when using large RNN units).
- Adjust EMBEDDING_DIM to experiment with the quality of learned character representations.

Tip: Start with these values, then fine-tune based on model performance and available computational resources!

In [ ]:
# Define model parameters
BATCH_SIZE = 64
BUFFER_SIZE = 10000
EMBEDDING_DIM = 256
RNN_UNITS = 1024
VOCAB_SIZE = len(vocab)

# Prepare dataset for training
dataset = (dataset.shuffle(BUFFER_SIZE)
           .batch(BATCH_SIZE, drop_remainder=True)
           .prefetch(tf.data.experimental.AUTOTUNE))

# LSTM-Based Model

Long Short-Term Memory (LSTM) is a type of Recurrent Neural Network (RNN) designed to capture long-range dependencies in sequential data. Unlike traditional RNNs, which struggle with vanishing gradients, LSTMs use gates (input, forget, and output gates) to regulate the flow of information, making them highly effective for text generation tasks.

This function defines an LSTM-based neural network for text generation.

**Function Overview:**
* [`tf.keras.layers.Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding): This is the input layer, consisting of a trainable lookup table that maps the numbers of each character to a vector with `embedding_dim` dimensions.
* [`tf.keras.layers.LSTM`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM): Our LSTM network, with size `units=rnn_units`.
* [`tf.keras.layers.Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense): The output layer, with `vocab_size` outputs.


<img src="https://raw.githubusercontent.com/aamini/introtodeeplearning/2019/lab1/img/lstm_unrolled-01-01.png" alt="Drawing"/>

In [ ]:
def build_lstm_model(vocab_size, embedding_dim, rnn_units, batch_size):
  """TO DO"""
  """
    Builds an LSTM-based text generation model.

    Parameters:
    - vocab_size (int): Number of unique characters in the vocabulary.
    - embedding_dim (int): Dimension of the word/character embeddings.
    - rnn_units (int): Number of units in the LSTM layer.
    - batch_size (int): Number of sequences processed in parallel.

    Returns:
    - tf.keras.Model: Compiled LSTM model.
    """


**Customization & Optimization**

These experiments are required to improve model performance. You must conduct the following experiments and document your findings:

- Increase RNN_UNITS : This enhances the model’s ability to recognize deeper patterns but may increase training time.
- Experiment with EMBEDDING_DIM : Adjusting this can improve the quality of character representation.
- Stack multiple LSTM layers : This can help the model understand text more effectively.

**Submission Requirements**

- Perform at least three trials varying the parameters above.
- Keep the most performing plots and outputs in your Jupyter Notebook.


# GRU-Based Model

What is GRU?

Gated Recurrent Units (GRU) are a simplified version of LSTMs that combine the forget and input gates into a single update gate. This makes GRUs:

- Faster and more efficient than LSTMs
- Perform well on shorter sequences
- Require fewer computational resources

**Model Comparison (LSTM vs. GRU)**

To evaluate the differences between LSTM and GRU models, compare them using:

- Model Size & Number of Parameters : Use model.summary() to check the number of trainable parameters in each model.
- Training Speed & Efficiency : Measure training time per epoch for both models.

In [ ]:
def build_gru_model(vocab_size, embedding_dim, rnn_units, batch_size):
    """TO DO"""
    """
    Builds a GRU-based text generation model.

    Parameters:
    - vocab_size (int): Number of unique characters in the vocabulary.
    - embedding_dim (int): Dimension of the word/character embeddings.
    - rnn_units (int): Number of units in the GRU layer.
    - batch_size (int): Number of sequences processed in parallel.

    Returns:
    - tf.keras.Model: Compiled GRU model.
    """

# Bidirectional Model


<img src="https://www.researchgate.net/publication/342646275/figure/fig4/AS:962238546464790@1606426955772/Comparison-between-LSTM-and-Bi-LSTM-networks-recreated-after-33.png" alt="Drawing"/>


**What is Bidirectional RNN?**

A Bidirectional Recurrent Neural Network (BiRNN) is an extension of a standard RNN that processes input sequences in both forward and backward directions. This means that at each time step, the model considers both past (left-to-right) and future (right-to-left) context, leading to better performance in many sequence-related tasks, including text generation.

**Why Use Bidirectional RNNs?**

Bidirectional RNNs are used because they capture dependencies in the input data from both directions, which is crucial for understanding context. This makes them particularly useful in tasks like language processing, where the meaning of a word can depend on both the words that come before and after it.

For example, in the sentence "The cat sat on the mat," the word "mat" is influenced by both the preceding words ("The cat sat on the") and what could potentially follow (e.g., "and looked at the mouse").

Example

Let's consider a sentence: "He opened the door."

Simple RNN: It processes the sentence from left to right, word by word:

He → opened → the → door
Each word is processed based on the previous word.

Bidirectional RNN: It processes the sentence in both directions:

Forward pass: He → opened → the → door

Backward pass: door → the → opened → He

The output at each time step is a combination of the information from both the forward and backward passes, allowing the network to understand the context better. For instance, knowing that "door" follows "the" helps confirm that "opened" likely refers to a physical action, not a metaphorical one.

**Choosing the Base RNN for Bidirectional Processing:**

One key flexibility of Bidirectional RNNs is that you can choose any recurrent architecture (LSTM, GRU, Simple RNN) as the base model.

- Bidirectional LSTM: Best for handling long-range dependencies.
- Bidirectional GRU: More computationally efficient than LSTM.
- Bidirectional Simple RNN: Less commonly used due to vanishing gradient issues.

Note: To determine the best base model for the Bidirectional RNN, you must evaluate the results from your previous experiments with different architectures (eg. LSTM and GRU).

Review previous results from training LSTM and GRU models.
Compare their performance in terms of:

- Training time per epoch
- Model size (number of parameters)
- Loss and accuracy on validation data
- Quality of generated text

Based on your findings, select the best-performing model as the base architecture for the Bidirectional RNN.


In [ ]:
def build_birnn_model(vocab_size, embedding_dim, rnn_units, batch_size, rnn_type="LSTM"):
    """TO DO"""
    """
    Builds a Bidirectional RNN-based text generation model.

    Parameters:
    - vocab_size (int): Number of unique characters in the vocabulary.
    - embedding_dim (int): Dimension of the word/character embeddings.
    - rnn_units (int): Number of units in the RNN layer.
    - batch_size (int): Number of sequences processed in parallel.
    - rnn_type (str): Type of RNN to use as the base (options: "LSTM", "GRU").

    Returns:
    - tf.keras.Model: Compiled Bidirectional RNN model.
    """

# Evaluating the Models

Model Selection

In [ ]:
# Choose model type
model_type = "LSTM"  # Change to "GRU" or "BiLSTM"

if model_type == "LSTM":
    model = build_lstm_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
elif model_type == "GRU":
    model = build_gru_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
elif model_type == "BiLSTM":
    model = build_birnn_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

print(f"Using {model_type} model for training.")

Compile & Train the Model

In [ ]:
# Define loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Compile the model
model.compile(optimizer='adam', loss=loss)

# Define checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

# Train the model
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

# Generate Text Using the Model

In [ ]:
class OneStepTextGenerator(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        """
        Initializes the OneStepTextGenerator.

        Parameters:
        - model: Trained text generation model (e.g., LSTM or GRU).
        - chars_from_ids: Function mapping character IDs to characters.
        - ids_from_chars: Function mapping characters to their respective IDs.
        - temperature (float): Controls randomness in text generation.
          -> Higher temperature (> 1.0): Produces more diverse and unpredictable text.
          -> Lower temperature (< 1.0): Makes the model more confident but results in repetitive text.
          -> Temperature = 1.0: Standard behavior without biasing randomness.
        """
        super().__init__()
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars
        self.temperature = temperature

    def generate_text(self, start_string, num_generate=1000):
        """
        Generates text one character at a time based on a starting string.

        Parameters:
        - start_string (str): Initial text prompt.
        - num_generate (int): Number of characters to generate.

        Returns:
        - str: Generated text.
        """
        # Convert start string to tensor representation
        input_eval = tf.expand_dims(self.ids_from_chars(tf.strings.unicode_split(start_string, 'UTF-8')), 0)
        text_generated = []

        # Initial hidden state (None allows automatic initialization)
        states = None

        for _ in range(num_generate):
            # Get model predictions and hidden state
            predictions, states = self.model(input_eval, states=states, return_state=True)

            # Adjust predictions using temperature
            predictions = predictions[:, -1, :] / self.temperature

            # Sample the next character ID from a probability distribution
            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

            # Use the predicted ID as next input
            input_eval = tf.expand_dims([predicted_id], 0)

            # Convert ID back to character and append to output
            text_generated.append(self.chars_from_ids(predicted_id).numpy().decode('utf-8'))

        return start_string + ''.join(text_generated)

# Create a text generator instance
text_generator = OneStepTextGenerator(model, chars_from_ids, ids_from_chars, temperature=0.8)

# Generate sample text
generated_text = text_generator.generate_text("ROMEO: ", num_generate=500)
print(generated_text)


# Evaluate Model Performance

**1- Perplexity (PP)**

TODO: Measures uncertainty in predicting next character for each model and leave the results in the Notebook.

***Lower values = better model***

In [ ]:
def perplexity(logits, labels):
    loss = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    return np.exp(np.mean(loss))

**2- Text Coherence & Fluency**

Subjective evaluation by examining generated text

**TODO: Generate at least 3 Samples (one from each model) and leave them in the Notebook.**

**3- BLEU Score**

TODO: Measures similarity between generated and real text for each model and leave the results in the Notebook.

In [ ]:
def compute_bleu(reference, generated):
    return sentence_bleu([list(reference)], list(generated))